## Guardian News Scraper

In [1]:
def guardian_news_scraper(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    author = soup.find_all('a', attrs={'rel' : 'author'})[0].get_text()
    date = soup.find_all('span', attrs={'class' : 'dcr-u0h1qy'})
    if len(date)==0:
        date = soup.find_all('div', attrs={'class' : 'dcr-18bzkx3'})
    date=date[0].get_text()
    
    headline = soup.find_all('h1', attrs={'class' : 'dcr-y70mar'})[0].get_text()
    standfirst = soup.find_all('div', attrs={'class' : 'dcr-1qt3dlu'})[0].get_text()
    
    text_elements = soup.find_all('p', attrs={'class' : 'dcr-1kas69x'})
    extracted_text = ' '.join([element.get_text() for element in text_elements])
    return {
        "headline":headline,
        "standfirst":standfirst,
        "author":author,
        "date":date,
        "text":extracted_text
    }

## Guardian Opinion Scraper

In [2]:
def guardian_opinion_scraper(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    author = soup.find_all('a', attrs={'rel' : 'author'})[0].get_text()
    date = soup.find_all('span', attrs={'class' : 'dcr-u0h1qy'}) + soup.find_all('div', attrs={'class' : 'dcr-18bzkx3'})+ soup.find_all('p', attrs={'class' : 'content__dateline'})
    
    date=date[0].get_text()    
    headline = soup.find_all('h1', attrs={'class' : 'dcr-y70mar'}) + soup.find_all('h1', attrs={'class' : 'dcr-1xaevyx'}) + soup.find_all('h1', attrs={'class' : 'content__headline'})
    headline=headline[0].get_text()    
    standfirst = soup.find_all('div', attrs={'class' : 'content__standfirst'}) + soup.find_all('div', attrs={'class' : 'dcr-1qt3dlu'}) + soup.find_all('div', attrs={'class' : 'dcr-xq41iu'}) 
    standfirst = standfirst[0].get_text()
    text_elements = soup.find_all('p', attrs={'class' : 'dcr-1kas69x'}) + soup.find_all('p', attrs={'class' : 'dcr-rysp4a'}) 
    extracted_text = ' '.join([element.get_text() for element in text_elements])
    return {
        "headline":headline,
        "standfirst":standfirst,
        "author":author,
        "date":date,
        "text":extracted_text
    }

In [3]:
def get_date_urls(url):
    response = requests.get(url)
    articles=[]
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')
    
        # Find all the articles on the page
        articles = soup.find_all('a',  attrs={'data-link-name' : 'article',"class":"fc-item__link"})
    return [a.get("href") for a in articles]

In [10]:
import json
import requests
from bs4 import BeautifulSoup
import os

In [261]:
jsons = []
year=2023
month="sep"
day="29"
opinion_url = f'https://www.theguardian.com/commentisfree/{year}/{month}/{day}/'
date=f"{year}_{month}_{day}"
ars = get_date_urls(opinion_url)
path = "/Users/kaanhamilton/JupyterProjects/truthchance/NewsScraping/data/guardian_articles/opinion/"
# Check whether the specified path exists or not
isExist = os.path.exists(path+date)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(f"{path+date}")
   print(f"{path+date} is created!")
for url in ars:
    name = "_".join(url.split("/")[-1].split("-"))
    
    if not os.path.exists(f"{path+date}/{name}.json"):
        try:
            article = guardian_opinion_scraper(url)
            if len(article["text"])>100:
                with open(f"{path+date}/{name}.json", "w") as outfile:
                    json.dump(article, outfile)
        except:
            print("Unknown Format")

Unknown Format
Unknown Format
Unknown Format
Unknown Format


In [6]:
import pandas as pd
from datetime import datetime
mapper={
    "01":"jan",
    "02":"feb",
    "03":"mar",
    "04":"apr",
    "05":"may",
    "06":"jun",
    "07":"jul",
    "08":"aug",
    "09":"sep",
    "10":"oct",
    "11":"nov",
    "12":"dec",
}
dates = pd.date_range(start="2020-01-01",end="2023-11-28")

In [7]:
dates = [str(date).split(" ")[0].split("-") for date in dates]

In [8]:
dates = [[date[0],mapper[date[1]],date[2]] for date in dates]

In [11]:
for date in dates:
    year=date[0]
    month=date[1]
    day=date[2]
    opinion_url = f'https://www.theguardian.com/commentisfree/{year}/{month}/{day}/'
    date=f"{year}_{month}_{day}"
    print(date)
    ars = get_date_urls(opinion_url)
    path = "/Users/kaanhamilton/JupyterProjects/truthchance/NewsScraping/data/guardian_articles/opinion/"
    # Check whether the specified path exists or not
    isExist = os.path.exists(path+year)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(f"{path+year}")
       print(f"{path+year} is created!")
    isExist = os.path.exists(path+year+"/"+month)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(f"{path+year}/{month}")
       print(f"{path+year}/{month} is created!")
    isExist = os.path.exists(path+year+"/"+month+"/"+day)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(f"{path+year}/{month}/{day}")
       print(f"{path+year}/{month}/{day} is created!")
    
    for url in ars:
        name = "_".join(url.split("/")[-1].split("-"))
        
        if not os.path.exists(f"{path+year}/{month}/{day}/{name}.json"):
            try:
                article = guardian_opinion_scraper(url)
                if len(article["text"])>100:
                    with open(f"{path+year}/{month}/{day}/{name}.json", "w") as outfile:
                        json.dump(article, outfile)
            except:
                print("Unknown Format")

2020_jan_01
/Users/kaanhamilton/JupyterProjects/truthchance/NewsScraping/data/guardian_articles/opinion/2020 is created!
/Users/kaanhamilton/JupyterProjects/truthchance/NewsScraping/data/guardian_articles/opinion/2020/jan is created!
/Users/kaanhamilton/JupyterProjects/truthchance/NewsScraping/data/guardian_articles/opinion/2020/jan/01 is created!
Unknown Format
Unknown Format
2020_jan_02
/Users/kaanhamilton/JupyterProjects/truthchance/NewsScraping/data/guardian_articles/opinion/2020/jan/02 is created!
Unknown Format
Unknown Format
Unknown Format
2020_jan_03
/Users/kaanhamilton/JupyterProjects/truthchance/NewsScraping/data/guardian_articles/opinion/2020/jan/03 is created!
Unknown Format
Unknown Format
Unknown Format
2020_jan_04
/Users/kaanhamilton/JupyterProjects/truthchance/NewsScraping/data/guardian_articles/opinion/2020/jan/04 is created!
2020_jan_05
/Users/kaanhamilton/JupyterProjects/truthchance/NewsScraping/data/guardian_articles/opinion/2020/jan/05 is created!
Unknown Format
202

In [242]:
path+date

'/Users/kaanhamilton/JupyterProjects/truthchance/NewsScraping/data/guardian_articles/opinion/2023_sep_28'

/Users/kaanhamilton/JupyterProjects/truthchance/NewsScraping/data/guardian_articles/opinion/2023_sep_28 is created!
